In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 14 09:10:01 2024

@author: abhis
"""

from langchain_community.graphs import Neo4jGraph
import matplotlib.pyplot as plt    
import pandas as pd
from neo4j import GraphDatabase
import os
import time   
import pandas as pd
from neo4j import GraphDatabase
import os
import time
from graphdatascience import GraphDataScience
wd = os.getcwd()
from dotenv import load_dotenv
load_dotenv()
database = os.environ.get('NEO4J_DATABASE')
gds = GraphDataScience(
    os.environ.get('NEO4J_URI'),
    auth = (os.environ.get('NEO4J_USERNAME'),os.environ.get('NEO4J_PASSWORD')),database = database
)       
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_ENDPOINT = os.getenv('OPENAI_BASE_URL') + '/embeddings'
kg = Neo4jGraph(
    url=os.environ.get('NEO4J_URI'), username=os.environ.get('NEO4J_USERNAME'), password=os.environ.get('NEO4J_PASSWORD'), database=database
)

In [3]:

#%%

try:
    gds.run_cypher("DROP INDEX overview_embeddings")
except:
    print("No index to drop")
query_index = (
            "CREATE VECTOR INDEX overview_embeddings "
            "FOR (m: Movie) ON (m.embedding) "
            "OPTIONS {indexConfig: { "
            "`vector.dimensions`: 1536, "
            "`vector.similarity_function`: 'cosine'}}"
        )            
        
gds.run_cypher(query_index)   

""


In [5]:
start_time = time.time()
node = """
MATCH (movie:Movie) WHERE movie.overview IS NOT NULL
WITH movie, genai.vector.encode(
    movie.overview, 
    "OpenAI", 
    {
      token: $openAiApiKey,
      endpoint: $openAiEndpoint
    }) AS vector
CALL db.create.setNodeVectorProperty(movie, "overview_embeddings", vector) 
return vector;
"""

result = gds.run_cypher(node, params={"openAiApiKey": OPENAI_API_KEY, "openAiEndpoint": OPENAI_ENDPOINT})
print(result)
print("TIME TAKEN FOR Generating embeddings :",time.time() - start_time)

                                              vector
0  [-0.013783346861600876, -0.04526415094733238, ...
1  [0.01771734282374382, -0.03553897887468338, -0...
TIME TAKEN FOR Generating embeddings : 0.6638326644897461
